In [1]:
!pip install tensorflow==2.8.0
!pip install gym
!pip install keras
!pip install keras-rl2


In [24]:
from gym import Env
from gym.spaces import Discrete,Box
import numpy as np
import random


In [8]:
class ShowerEnv(Env):#self.state=temperature
  def __init__(self):
    self.action_space=Discrete(3)

    self.observation_space = Box(low=np.array([0]), high=np.array([100]))
    self.state=38+random.randint(-3,3)
    self.shower_length=60
  def step(self,action):
    self.state+=action-1
    self.shower_length-=1
    if self.state>=38 and self.state<=39:
      reward=1
    else:
      reward=-1
    if self.shower_length<=0:
      done=True
    else:
      done=False

    self.state+=random.randint(-1,1)

    info={}
    return self.state,reward,done,info

  def render(self):
    pass

  def reset(self):
    self.state = 38 + random.randint(-3,3)
    self.shower_length = 60
    return self.state



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
env = ShowerEnv()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [10]:
env.observation_space.sample()

array([82.01137], dtype=float32)

In [11]:

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-60
Episode:2 Score:-58
Episode:3 Score:-34
Episode:4 Score:-36
Episode:5 Score:-60
Episode:6 Score:-60
Episode:7 Score:-54
Episode:8 Score:-32
Episode:9 Score:-56
Episode:10 Score:-36


In [12]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:

states = env.observation_space.shape
actions = env.action_space.n

In [14]:
actions

3

In [15]:
def build_model(states, actions):
  model = Sequential()
  model.add(Dense(24, activation='relu', input_shape=states))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model

In [5]:
del model

NameError: name 'model' is not defined

In [16]:
model = build_model(states, actions)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [2]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [14]:
!pip install tensorflow-keras

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ERROR: Could not find a version that satisfies the requirement tensorflow-keras (from versions: none)
ERROR: No matching distribution found for tensorflow-keras


In [5]:
import keras

In [18]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [19]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:39 - reward: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   35/10000 [..............................] - ETA: 3:05 - reward: -0.8286

/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch

10000/10000 [==============================] - 75s 7ms/step - reward: -0.7694
166 episodes - episode_reward: -46.145 [-60.000, -8.000] - loss: 1.720 - mae: 8.661 - mean_q: -11.929

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: -0.8064
167 episodes - episode_reward: -48.431 [-60.000, -8.000] - loss: 2.886 - mae: 13.285 - mean_q: -19.111

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 80s 8ms/step - reward: -0.7636
167 episodes - episode_reward: -45.784 [-60.000, 16.000] - loss: 3.096 - mae: 13.967 - mean_q: -20.126

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 75s 8ms/step - reward: -0.7632
166 episodes - episode_reward: -45.880 [-60.000, -12.000] - loss: 2.558 - mae: 12.779 - mean_q: -18.299

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -0.7734
done, took 381.526 seconds


In [20]:

scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -56.000, steps: 60
Episode 3: reward: -52.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -58.000, steps: 60
Episode 6: reward: -56.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -54.000, steps: 60
Episode 9: reward: -58.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -52.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -56.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: -54.000, steps: 60
Episode 20: reward: -58.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -58.000, steps: 60
Episode 23: reward: -58.000, steps: 60
Episode 24: reward: -56.000, steps: 60
Episode 25: reward: -60.000, steps: 60
Episo